### In this notebook we look into the multiscale analysis of the Sinkhorn algorithm.

In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Helper Functions

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( plt, x, col, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors, c = col, cmap = 'plasma', linewidths = 2 )

### Functions to Generate data points

In [ ]:
def randomsampledata( N ):
  x = []
  y = []
  for i in range(len(N)):
    x.append( np.random.rand( 2, N[i] ) - 0.5 )
    theta = 2 * np.pi * np.random.rand( 1, N[i] )
    r = 0.8 + 0.2 * np.random.rand( 1, N[i] )
    y.append( np.vstack( ( np.cos(theta) * r, np.sin(theta) * r ) ) )
  return x, y

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def clusteredsampling( N ):
  x_sample = []
  y_sample = []
  label_x = []
  label_y = []
  N = np.sort( N ) 
  x = np.random.rand( 2, N[-1] ) - 0.5
  theta = 2 * np.pi * np.random.rand( 1, N[ - 1 ] )
  r = 0.8 + 0.2 * np.random.rand( 1, N[ - 1 ] )
  y = np.vstack( ( np.cos(theta) * r, np.sin(theta) * r ) )
  x_sample.append( x.T )
  y_sample.append( y.T )
  n = len( N )
  for i in range( n - 1, 0, - 1 ):
      # if i!=n-1:
      x = x_sample[ - 1 ]
      y = y_sample[ - 1 ]
      # N[i-1]=4, x resolution =16
      kmeans_x = KMeans( n_clusters = N[i - 1], random_state = i ).fit( x )
      kmeans_y = KMeans( n_clusters = N[i  - 1], random_state = i ).fit( y )
      center_x = kmeans_x.cluster_centers_
      center_y = kmeans_y.cluster_centers_
      label_x.append( kmeans_x.labels_ )
      label_y.append( kmeans_y.labels_ )
      x_sample.append( center_x )
      y_sample.append( center_y )
  # end for
  return x_sample[::-1], y_sample[::-1], label_x[::-1], label_y[::-1], N

In [ ]:
N = [ 4, 16, 64, 256, 1024, 4096 ]
x, y, label_x, label_y, N = clusteredsampling( N )

In [ ]:
for i in label_x:
  print( ( len(i), i ) )
# end for

### Voronoi Plot

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d

In [ ]:
for i in range( len(N) ):
  _, ( ax1, ax2 ) = plt.subplots( figsize = ( 8, 5 ), ncols = 2 )
  vorx = Voronoi( x[i] )
  fig = voronoi_plot_2d( vorx, ax = ax1, show_vertices = False )
  vory = Voronoi( y[i] )
  fig = voronoi_plot_2d( vory, ax = ax2, show_vertices = False )  
  ax1.set_title( "Square Resolution = "+str(x[i].shape[0]) )
  ax2.set_title( "Annulus Resolution = "+str(y[i].shape[0]) )
  plt.show();
# end for

### Inflate Function

In [ ]:
def inflate( x, labelpts ):
  
  mat = np.zeros( ( len(labelpts), len(x) ) )
  for i in range((len(labelpts))):
    mat[i][ labelpts[i] ] = 1
  # end for
  return np.dot( mat, x )

### Importing Sinkhorn

In [ ]:
import computational_OT

### Experiment

In [ ]:
epsilons = [ 0.5, .06, .06, .06, .06, .06 ]
# Sinkhorn
print("Sinkhorn.... ")
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
finflated = []
ginflated = []
multiscale_f = []
multiscale_g = []
for i in range(len(N)):
  xi, yi = x[i].T, y[i].T
  #Cost matrix
  C = distmat( xi, yi )
  # a and b
  a = normalize( np.ones( N[i] ) )
  b = normalize( np.ones( N[i] ) )
  #Epsilon
  epsilon = epsilons[i]
  #Kernel
  K = np.exp( - C/epsilon )
  print( "Doing for ",N[i] )
  print( " |- Iterating" )
  plt.figure( figsize = ( 5, 5 ) )
  plotp( plt,xi, 'b')
  plotp( plt,yi, 'r' )
  plt.axis("off")
  plt.xlim( np.min( yi[0,:] ) - .1, np.max( yi[0,:]) + .1 )
  plt.ylim( np.min( yi[1,:] ) - .1, np.max( yi[1,:] ) + .1 )
  plt.show()
  #Inflating
  if i == 0:
    u = a
    v = b
  else:
    #Optimized f&g for N-1
    f_coarse = multiscale_f[-1]
    g_coarse = multiscale_g[-1]

    # Inflated f&g to initialise for optimizing for N
    f_inflated = inflate( f_coarse,label_x[i-1] )
    finflated.append( f_inflated )
    u = np.exp( f_inflated/epsilon ) 

    g_inflated = inflate( g_coarse, label_y[i-1] )
    ginflated.append( g_inflated )
    v = np.exp( g_inflated/epsilon )
  # end if   
  u = u
  v = v
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          epsilon )
  out = Optimizer._update()
  results_Sinkhorn.append( out )
  end = time.time() 
  times_Sinkhorn.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/epsilon )
  K = np.exp( - C/epsilon )
  v_opt =  np.exp( out['potential_g']/epsilon )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP.append( P_opt )  # Distance between P matrices of Linear P and Sinkhorn P
  multiscale_f.append( out['potential_f'] )  
  multiscale_g.append( out['potential_g'] ) 
#end for

### Convergence Plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1$" )
for result in results_Sinkhorn:
  plt.plot( np.asarray(result['error_a']), linewidth = 2)
# end for
plt.yscale( 'log')
plt.legend( [ str(i) for i in N ], loc = "upper right" )
plt.subplot( 2, 1, 2 )
plt.title( "$||P^T 1 -b||_1$" )
for result in results_Sinkhorn:
  plt.plot( np.asarray(result['error_b']), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( [ str(i) for i in N ], loc = "upper right" )
plt.show()

### Objective Function

In [ ]:
# targets=[]
# penalizations=[]

# for result in results_Sinkhorn:
#   objective = result['objectives']
#   target = np.asarray( [x for (x,y) in objective] ).flatten()
#   penalization = np.asarray( [y for (x,y) in objective] ).flatten()
#   targets.append( target )
#   penalizations.append( penalization )

# plt.figure(figsize = (20,7))
# plt.title("$Objectivefunction$")
# for target in targets:  
#   plt.plot( target, linewidth = 2)
# plt.legend([str(i) for i in N],loc="upper right")
# plt.show()

# plt.figure(figsize = (20,7))
# plt.title("$Penalizations$")
# for penalization in penalizations:  
#   plt.plot( penalization, linewidth = 2)
# plt.legend([str(i) for i in N],loc="upper right")
# plt.show()

### f&g map plots

In [ ]:
from locale import MON_1


for i in range(0, len(N) - 1):
  print("\n")
  fig, ax = plt.subplots( figsize = ( 20, 5 ), nrows = 1, ncols = 4 )

  dot_scale = 2 * 2**( len(N) - i )

  xi, yi = x[i + 1].T, y[i + 1].T
  f   = multiscale_f[ i + 1]
  m1  = f.mean()
  im1 = plotp( ax[0], xi, f, scale = dot_scale, edgecolors = ( 0, 0, 0, 0 ) )
  fig.colorbar( im1, ax = ax[0] )

  f   = finflated[i]
  m2  = f.mean()
  im2 = plotp( ax[1], xi, f, scale = dot_scale, edgecolors = ( 0, 0, 0, 0 ) )
  fig.colorbar( im2, ax = ax[1] )

  g = multiscale_g[i + 1]
  m3 = g.mean()
  im3 = plotp( ax[2], yi, g, scale = dot_scale, edgecolors = ( 0, 0, 0, 0 ) )
  fig.colorbar( im3, ax = ax[2] )

  g  = ginflated[i]
  m4 = g.mean()
  im4 = plotp( ax[3], yi, g, scale = dot_scale, edgecolors = ( 0, 0, 0, 0 ) )
  fig.colorbar( im3, ax = ax[3] )

  # print( "Stats f:")
  # print( m1 )
  # print( m2 )
  # print( m1-m2)
  # print( m1/m2)

  # print( "Stats g:")
  # print( m3 )
  # print( m4 )
  # print( m3-m4)
  # print( m3/m4)

  # print( "Mixed")
  # print( m1 + m3 )
  # print( m2 + m4 )


  ax[0].set_title( "Optimized f maps for N = "+str(N[i + 1] ) )
  ax[1].set_title( "Inflated f maps for N = "+str(N[i + 1] ) )

  ax[2].set_title( "Optimized g maps for N = "+str(N[i + 1] ) )
  ax[3].set_title( "Inflated g maps for N = "+str(N[i + 1] ) )
# end for
  

In [ ]:
# Comparing Sinkhorn solutions to inflated versions from coarser scale
assert( len(finflated) == len(multiscale_f) - 1 )
scales_count = len(finflated)
f_error = []
g_error = []
for i in range( scales_count ):
  solution = multiscale_f[i + 1]
  coarse   = finflated[i]
  f_error.append( np.linalg.norm( solution - coarse)/len(solution) )
  solution = multiscale_g[i + 1]
  coarse   = ginflated[i]
  g_error.append( np.linalg.norm( solution - coarse)/len(solution) )
# end for
# Plotting
plt.figure( figsize = ( 5, 5 ) )
plt.plot( range(scales_count), f_error,'-bD', label = 'Errors for f' )
plt.plot( range(scales_count), g_error, '-rD', label = 'Errors for g' )
plt.xlabel( 'Scale ')
plt.legend()
plt.show()

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "Time Plot" )
plt.plot( list(range(1, len(times_Sinkhorn) + 1)), times_Sinkhorn, '-gD' )
# plt.yscale( 'log')
plt.ylabel( "Time in second" )
plt.xlabel( "Dimensions of samples drawn" )
plt.xticks( list(range(1, len(times_Sinkhorn) + 1 ) ), [ str(i) for i in N ] )
plt.legend( [ "Sinkhorn" ], loc = "upper left" )
plt.show()